In [5]:
%run GUIanaliza.py
%run velikaMestaEvrope.py
import pandas as pd
import matplotlib.pyplot as plt

# Analiza izbrane države
#### PROJEKTNA NALOGA PRI PROGRAMIRANJU 1
Podatki so bili zajeti iz sledečih spletnih strani:
http://www.weatherbase.com/
država, mesto,  id, povprečne temperature po posameznih mesecih;
http://www.weatherbase.com/weather/country.php3?r=EUR&regionname=Europe:
 seznam Evropskih držav z njihovimi prestolnicami (v drugem delu).

V prvem delu analize mora uporabnik sam izbrati državo, ki jo želi analizirati. Nato vnese še izbrani mesec in temperaturo. S tem programom nato lahko poišče mesto, ki ima v izbranem mesecu temperaturo najbližjo vnešeni.
Najprej prosim zaženi spodnjo funkcijo:

In [6]:
izbrani_mesec, ime_datotekeOsn, ime_datotekeRaz = zazeni_analizaGUI()

[['Andorra La Vella', 'Andorra', '342955', '32.4,33.8,37.4,41.4,48.4,55.4,61.5,60.8,56.3,48.2,38.7,33.8']]

    izbrana drzava: Andorra 

    izbrani mesec: januar 

    izbrana temperatura: 14°C 

    


In [ ]:
osnovni_df = pd.read_csv(ime_datotekeOsn, encoding='latin1')
razlikaTemperatur_df = pd.read_csv(ime_datotekeRaz, encoding='latin1')
priporoceni_df = pd.read_csv("priporoceni.txt", encoding='latin1')

In [43]:
osnovni_df[:]

,MESTO,Drzava,sifra,januar,februar,marec,april,maj,juni,juli,avgust,september,oktober,november,december,LETNO POVPRECJE,razlika od vnosa
0,Andorra La Vella,Andorra,342955,0.2,1.0,3.0,5.2,9.1,13.0,16.4,16.0,13.5,9.0,3.7,1.0,7.0,11.8
1,Soldeu,Andorra,592658,-2.8,-2.8,-1.1,0.6,5.0,8.9,12.2,11.7,9.4,5.0,0.6,-2.2,3.0,9.2


In [ ]:
mesec_df = razlikaTemperatur_df[['MESTO', 'LETNO POVPRECJE', 'razlika od vnosa']]



#drzave50 = naj50[['leto', 'mesto', 'ime_tekmovalca', 'država']]
#ekipe50 = naj50[['leto', 'mesto', 'ime_tekmovalca', 'ekipa']]

#drzave10 = drzave50[drzave50.mesto <= 10]
#ekipe10 = ekipe50[ekipe50.mesto <= 10]

#p1 = pd.DataFrame(drzave10['država'].value_counts()[:10])
#p1.columns = ['št uvrstitev med prvih 10']
#p1